In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random

In [2]:
path = os.path.dirname(os.getcwd())
data_path = os.path.join(path, 'data','web-Stanford.txt', 'web-Stanford.txt')
data_path

'c:\\Users\\Linyi Zhang\\Desktop\\data_mining\\HW3\\Datamining_HW3\\data\\web-Stanford.txt\\web-Stanford.txt'

In [3]:
def reservoir_sampling(filename, sample_size):
    reservoir = []
    with open(filename, 'r') as file:
        # Skip the header lines
        for _ in range(4):
            next(file)
        
        line_number = 0
        for line in file:
            line_number += 1
            if line.strip():  # Check if the line is not empty
                if len(reservoir) < sample_size:
                    reservoir.append(line.strip())
                else:
                    # Generate a random index from 0 to line_number (inclusive)
                    random_index = random.randint(0, line_number)
                    if random_index < sample_size:
                        reservoir[random_index] = line.strip()
    
    return reservoir



In [4]:
filename = data_path
sample_size = 10  # Set your desired sample size here
sample = reservoir_sampling(filename, sample_size)

# Print the sampled edges
print("Sampled edges:")
for edge in sample:
    print(edge)


Sampled edges:
17036	11495
216321	234704
183982	83427
195161	119221
15845	170347
124610	267421
249778	69358
153046	100340
21361	232570
144087	160065
